### Data Cleaning for Glacier Retreat Models

In [168]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import balanced_accuracy_score
from pathlib import Path
import numpy as np

In [169]:
# List Data Files in "resources"
import os
# Specify the folder path
folder_path = 'resources'
# List all files in the folder
filenames = os.listdir(folder_path)
print(filenames)

['00_Glacier_Area_Retreat_1979_2023.csv', '01_avg_arctic_land_ocean_temps_1972_2024.csv', '02_Pop_Farming_Worldbank_Categories_1972_2022.csv', '03_CO2_Mean_and_Increase_1979_2023.csv', '04_dT_by_country_1972_2022.csv', '05_Forestation_Perc_Change_By_Country_1990_2021.csv', '06_Global_SeaLevel_Rise_1972_2022.csv', 'describe_output_df_forestation.csv', 'describe_output_dT_country.csv', 'null_df_forestation.csv', 'null_dT_country.csv', 'null_dT_country_r0.csv', 'parameters_1.csv', 'parameters_2.csv', 'parameters_3.csv', 'parameters_4.csv', 'sample_data_2.csv']


In [170]:
# Establish dataframes for each data set
df_glacier_retreat = pd.read_csv('resources/00_Glacier_Area_Retreat_1979_2023.csv')
df_arctic_land_ocean_temps = pd.read_csv('resources/01_avg_arctic_land_ocean_temps_1972_2024.csv')
df_pop_farming_country = pd.read_csv('resources/02_Pop_Farming_Worldbank_Categories_1972_2022.csv')
df_CO2 = pd.read_csv('resources/03_CO2_Mean_and_Increase_1979_2023.csv')
df_dT_country = pd.read_csv('resources/04_dT_by_country_1972_2022.csv')
df_forestation = pd.read_csv('resources/05_Forestation_Perc_Change_By_Country_1990_2021.csv')
df_searise = pd.read_csv('resources/06_Global_SeaLevel_Rise_1972_2022.csv')

In [171]:
# Review dataframe start date and end date.
dataframes = [df_glacier_retreat, df_arctic_land_ocean_temps, df_pop_farming_country, df_CO2, df_dT_country, df_forestation, df_searise]
for df in dataframes:
    display()
    display(df.head(1), df.tail(1))

,year,glacier_extent,glacier_retreat
0,1979,6.903,0


,year,glacier_extent,glacier_retreat
44,2023,4.37,0


,Year,Arctic_DegC_Anomaly
0,1972,0.07


,Year,Arctic_DegC_Anomaly
52,2024,1.21


,Year,SP.URB.TOTL.IN.ZS,SP.URB.TOTL,SP.URB.GROW,SP.POP.TOTL,SP.POP.GROW,EN.URB.MCTY.TL.ZS,SE.PRM.CMPT.ZS,BX.KLT.DINV.WD.GD.ZS,AG.YLD.CREL.KG
0,1972,36.920174,1418592514,2.600355,3843630361,2.008512,15.864215,72.803391,0.423371,1953.469567


...

,Year,AFG,ALB,DZA,ASM,AND,AGO,ATG,ARG,AUS,...,USA,VIR,URY,VUT,VEN,VNM,PSE,WLD,ZMB,ZWE
50,2022,2.012,1.518,1.688,1.256,3.243,1.212,0.77,0.643,0.754,...,1.217,0.894,0.382,1.479,0.533,1.033,1.074,1.394,0.686,-0.49


,Year,ABW,AFE,AFG,AFW,AGO,ALB,AND,ARB,ARE,...,VIR,VNM,VUT,WLD,WSM,XKX,YEM,ZAF,ZMB,ZWE
0,1991,0,-0.151253,0,-0.118162,-0.124663,-0.071168,0,-0.018069,0.090721,...,-1.162857,0.739851,0,0.0,-0.170318,0,0,-0.030006,-0.048158,-0.11909


,Year,ABW,AFE,AFG,AFW,AGO,ALB,AND,ARB,ARE,...,VIR,VNM,VUT,WLD,WSM,XKX,YEM,ZAF,ZMB,ZWE
30,2021,0,-0.219059,0,-0.084741,-0.445229,0.0,0,-0.017774,0.0,...,0.428571,0.24216,0,0.000144,-0.172662,0,0,-0.030006,-0.253187,-0.11909


,Year,Global sea level average Church and White (2011) and UHSLC,ann_d_GSL_mm,Total_GSL_mm
0,1972,-49.322704,3.008697,133.991356


,Year,Global sea level average Church and White (2011) and UHSLC,ann_d_GSL_mm,Total_GSL_mm
50,2022,69.66,6.37,252.97406


In [172]:
# 'glacier_retreat' column in that dataframe is 1 = no retreat (longer), 0 = retreat
# This is our y column for the models. Review the data. 
display(df_glacier_retreat.dtypes)
display(df_glacier_retreat["glacier_retreat"].value_counts())
display(df_glacier_retreat['glacier_retreat'].unique().tolist()
)

year                 int64
glacier_extent     float64
glacier_retreat      int64
dtype: object

glacier_retreat
0    26
1    19
Name: count, dtype: int64

[0, 1]

### Parameter Set 1:  Glacier Retreat, CO2 Increase, Arctic Land Ocean Temps, and Global Sea Rise

In [173]:
# For the first model data set, the X set with be Arctic Land and Ocean Temps
# data, the CO2 data, and the searise data.  Review and concatenate data for
# common years.

# Start all data at 1979 to match glacier_retreat outcome.  
# df_arctic_land_ocean_temps and df_CO2
df_arctic_land_ocean_temps.head(10)


,Year,Arctic_DegC_Anomaly
0,1972,0.07
1,1973,0.15
2,1974,0.01
3,1975,0.02
4,1976,0.00
5,1977,0.22
6,1978,0.08
7,1979,0.12
8,1980,0.31
9,1981,0.32


In [174]:
# Filter for all rows 1979 and up
df_arctic_land_ocean_temps_r1 = df_arctic_land_ocean_temps[df_arctic_land_ocean_temps['Year'] >= 1979]
df_arctic_land_ocean_temps_r1 = df_arctic_land_ocean_temps_r1.reset_index(drop=True)
df_arctic_land_ocean_temps_r1.tail()

,Year,Arctic_DegC_Anomaly
41,2020,0.91
42,2021,0.90
43,2022,0.90
44,2023,1.18
45,2024,1.21


In [175]:
# Filter for all rows 1979 and up
df_searise_r1 = df_searise[df_searise['Year'] >= 1979]
df_searise_r1 = df_searise_r1.reset_index(drop=True)
df_searise_r1.tail()

,Year,Global sea level average Church and White (2011) and UHSLC,ann_d_GSL_mm,Total_GSL_mm
39,2018,52.083272,0.195536,235.397332
40,2019,60.980739,3.487047,244.294800
41,2020,65.999737,-1.376702,249.313797
42,2021,63.290000,1.546960,246.604060
43,2022,69.660000,6.370000,252.974060


In [178]:
# Combine the dataframes into one dataframe
df_env_parameters_1 = pd.concat([df_glacier_retreat, df_arctic_land_ocean_temps_r1, 
                            df_CO2, df_searise_r1], axis="columns",join="inner")
df_env_parameters_1.head()

,year,glacier_extent,glacier_retreat,Year,Arctic_DegC_Anomaly,Year,Mean_CO2_PPM,Annual_Increase_PPM,Year,Global sea level average Church and White (2011) and UHSLC,ann_d_GSL_mm,Total_GSL_mm
0,1979,6.903,0,1979,0.12,1979,336.85,2.14,1979,-48.978593,1.234363,134.335467
1,1980,7.544,1,1980,0.31,1980,338.91,1.71,1980,-39.865623,1.049035,143.448437
2,1981,6.903,0,1981,0.32,1981,340.11,1.15,1981,-28.923696,2.591535,154.390364
3,1982,7.166,1,1982,0.18,1982,340.86,1.00,1982,-31.404629,0.234687,151.909431
4,1983,7.221,1,1983,0.23,1983,342.53,1.83,1983,-26.608179,-1.269578,156.705881


In [179]:
# Drop columns not needed, keeping glacier retreat, change in arctic sea and land
# temperature, increase in CO2(ppm), and global sea level rise.

df_env_parameters_1 = df_env_parameters_1.drop(df_env_parameters_1.columns[[3]], axis=1)
df_env_parameters_1.head()

,year,glacier_extent,glacier_retreat,Arctic_DegC_Anomaly,Mean_CO2_PPM,Annual_Increase_PPM,Global sea level average Church and White (2011) and UHSLC,ann_d_GSL_mm,Total_GSL_mm
0,1979,6.903,0,0.12,336.85,2.14,-48.978593,1.234363,134.335467
1,1980,7.544,1,0.31,338.91,1.71,-39.865623,1.049035,143.448437
2,1981,6.903,0,0.32,340.11,1.15,-28.923696,2.591535,154.390364
3,1982,7.166,1,0.18,340.86,1.00,-31.404629,0.234687,151.909431
4,1983,7.221,1,0.23,342.53,1.83,-26.608179,-1.269578,156.705881


In [180]:
# Drop columns not needed, keeping glacier retreat, change in arctic sea and land
# temperature, increase in CO2(ppm), and global sea level rise.
df_env_parameters_1 = df_env_parameters_1.drop(df_env_parameters_1.columns[[1,4,6,8]], axis=1)


In [181]:
# View final Parameters 1 dataframe
df_env_parameters_1

,year,glacier_retreat,Arctic_DegC_Anomaly,Annual_Increase_PPM,ann_d_GSL_mm
0,1979,0,0.12,2.14,1.234363
1,1980,1,0.31,1.71,1.049035
2,1981,0,0.32,1.15,2.591535
3,1982,1,0.18,1.00,0.234687
4,1983,1,0.23,1.83,-1.269578
5,1984,0,0.17,1.23,1.070493
6,1985,1,0.04,1.65,-1.852663
7,1986,1,0.17,1.02,0.432997
8,1987,0,0.48,2.65,-0.211762
9,1988,1,0.36,2.16,1.906937


In [182]:
# Write Parameters 1 to file
df_env_parameters_1.to_csv('resources/env_parameters_1.csv', index=False)

## Parameter Set 2: Population and Farming Parameters (1979 to 2022)

Source:  https://data.worldbank.org/topic/climate-change

Parameters Legend:

Urban population (% of total population)	SP.URB.TOTL.IN.ZS

Urban population	SP.URB.TOTL 

Urban population growth (annual %)	SP.URB.GROW

Population, total	SP.POP.TOTL

Population growth (annual %)	SP.POP.GROW

Population in urban agglomerations of more than 1 million (% of total population)	EN.URB.MCTY.TL.ZS

Primary completion rate, total (% of relevant age group)	SE.PRM.CMPT.ZS

Foreign direct investment, net inflows (% of GDP)	BX.KLT.DINV.WD.GD.ZS

Cereal yield (kg per hectare)	AG.YLD.CREL.KG


In [183]:
# Review the dataframe
display(df_pop_farming_country.head())
df_pop_farming_country.tail()

,Year,SP.URB.TOTL.IN.ZS,SP.URB.TOTL,SP.URB.GROW,SP.POP.TOTL,SP.POP.GROW,EN.URB.MCTY.TL.ZS,SE.PRM.CMPT.ZS,BX.KLT.DINV.WD.GD.ZS,AG.YLD.CREL.KG
0,1972,36.920174,1418592514,2.600355,3843630361,2.008512,15.864215,72.803391,0.423371,1953.469567
1,1973,37.153428,1455932571,2.632190,3920044841,1.988081,15.969004,72.777977,0.459476,1985.437886
2,1974,37.417814,1494670627,2.660704,3995920225,1.935574,16.086386,76.349892,0.485732,1966.867747
3,1975,37.649413,1531821571,2.485561,4070060140,1.855390,16.169538,78.539352,0.465998,2080.389637
4,1976,37.909994,1570112560,2.499703,4143135434,1.795435,16.288070,77.747963,0.322544,2088.677693


,Year,SP.URB.TOTL.IN.ZS,SP.URB.TOTL,SP.URB.GROW,SP.POP.TOTL,SP.POP.GROW,EN.URB.MCTY.TL.ZS,SE.PRM.CMPT.ZS,BX.KLT.DINV.WD.GD.ZS,AG.YLD.CREL.KG
46,2018,55.187622,4227011393,1.938597,7661177849,1.109496,23.795219,87.743073,0.999459,4035.038575
47,2019,55.630612,4306311635,1.876036,7742724795,1.064418,24.002074,87.487778,2.077462,4131.819598
48,2020,56.064731,4383953178,1.802971,7821271846,1.014463,24.222277,87.606293,1.402679,4100.768039
49,2021,56.478131,4454512575,1.609492,7888963821,0.865486,24.477826,88.389748,2.321421,4152.232718
50,2022,56.901955,4523589075,1.550708,7951595433,0.793914,24.748090,91.138542,1.743429,4182.382970


In [184]:
# Review the data types associated with the columns
df_pop_farming_country.dtypes

Year                      int64
SP.URB.TOTL.IN.ZS       float64
SP.URB.TOTL               int64
SP.URB.GROW             float64
SP.POP.TOTL               int64
SP.POP.GROW             float64
EN.URB.MCTY.TL.ZS       float64
SE.PRM.CMPT.ZS          float64
BX.KLT.DINV.WD.GD.ZS    float64
AG.YLD.CREL.KG          float64
dtype: object

In [185]:
# Check for outliers and similar anomolies
df_pop_farming_country.describe()

,Year,SP.URB.TOTL.IN.ZS,SP.URB.TOTL,SP.URB.GROW,SP.POP.TOTL,SP.POP.GROW,EN.URB.MCTY.TL.ZS,SE.PRM.CMPT.ZS,BX.KLT.DINV.WD.GD.ZS,AG.YLD.CREL.KG
count,51.000000,51.000000,5.100000e+01,51.000000,5.100000e+01,51.000000,51.000000,51.000000,51.000000,51.000000
mean,1997.000000,46.138386,2.789027e+09,2.351807,5.886155e+09,1.475617,19.730133,82.665531,1.740538,3049.398334
std,14.866069,6.084091,9.408145e+08,0.305587,1.245003e+09,0.304162,2.712119,4.258924,1.293217,657.745298
min,1972.000000,36.920174,1.418593e+09,1.550708,3.843630e+09,0.793914,15.864215,72.777977,0.322544,1953.469567
25%,1984.500000,41.010836,1.971334e+09,2.190815,4.808462e+09,1.238699,17.289818,80.052399,0.575679,2638.947812
50%,1997.000000,45.537205,2.683936e+09,2.325936,5.896175e+09,1.453619,19.374677,81.884422,1.312064,2987.785970
75%,2009.500000,51.331573,3.555287e+09,2.617627,6.927752e+09,1.755283,22.039747,87.519775,2.652450,3551.192029
max,2022.000000,56.901955,4.523589e+09,2.868308,7.951595e+09,2.008512,24.748090,91.138542,5.292705,4182.382970


In [186]:
# Check for non-null entries
df_pop_farming_country.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Year                  51 non-null     int64  
 1   SP.URB.TOTL.IN.ZS     51 non-null     float64
 2   SP.URB.TOTL           51 non-null     int64  
 3   SP.URB.GROW           51 non-null     float64
 4   SP.POP.TOTL           51 non-null     int64  
 5   SP.POP.GROW           51 non-null     float64
 6   EN.URB.MCTY.TL.ZS     51 non-null     float64
 7   SE.PRM.CMPT.ZS        51 non-null     float64
 8   BX.KLT.DINV.WD.GD.ZS  51 non-null     float64
 9   AG.YLD.CREL.KG        51 non-null     float64
dtypes: float64(7), int64(3)
memory usage: 4.1 KB


In [187]:
# Filter for all rows 1979 and up
df_pop_farming_country_r1 = df_pop_farming_country[df_pop_farming_country['Year'] >= 1979]
df_pop_farming_country_r1 = df_pop_farming_country_r1.reset_index(drop=True)
df_pop_farming_country_r1.tail()

,Year,SP.URB.TOTL.IN.ZS,SP.URB.TOTL,SP.URB.GROW,SP.POP.TOTL,SP.POP.GROW,EN.URB.MCTY.TL.ZS,SE.PRM.CMPT.ZS,BX.KLT.DINV.WD.GD.ZS,AG.YLD.CREL.KG
39,2018,55.187622,4227011393,1.938597,7661177849,1.109496,23.795219,87.743073,0.999459,4035.038575
40,2019,55.630612,4306311635,1.876036,7742724795,1.064418,24.002074,87.487778,2.077462,4131.819598
41,2020,56.064731,4383953178,1.802971,7821271846,1.014463,24.222277,87.606293,1.402679,4100.768039
42,2021,56.478131,4454512575,1.609492,7888963821,0.865486,24.477826,88.389748,2.321421,4152.232718
43,2022,56.901955,4523589075,1.550708,7951595433,0.793914,24.748090,91.138542,1.743429,4182.382970


In [188]:
# Combine the dataframes into one dataframe
df_pop_farm_parameters_2 = pd.concat([df_glacier_retreat, df_pop_farming_country_r1], axis="columns",join="inner")
display(df_pop_farm_parameters_2.head(3))
df_pop_farm_parameters_2.tail(3)

,year,glacier_extent,glacier_retreat,Year,SP.URB.TOTL.IN.ZS,SP.URB.TOTL,SP.URB.GROW,SP.POP.TOTL,SP.POP.GROW,EN.URB.MCTY.TL.ZS,SE.PRM.CMPT.ZS,BX.KLT.DINV.WD.GD.ZS,AG.YLD.CREL.KG
0,1979,6.903,0,1979,38.915541,1698369060,2.837103,4365802882,1.770476,16.682243,77.901047,0.428142,2331.575510
1,1980,7.544,1,1980,39.330138,1746580426,2.838686,4442416674,1.754861,16.822785,80.020821,0.490715,2297.408450
2,1981,6.903,0,1981,39.755159,1796677728,2.868308,4520993169,1.768778,16.929199,80.550163,0.617123,2438.721357


,year,glacier_extent,glacier_retreat,Year,SP.URB.TOTL.IN.ZS,SP.URB.TOTL,SP.URB.GROW,SP.POP.TOTL,SP.POP.GROW,EN.URB.MCTY.TL.ZS,SE.PRM.CMPT.ZS,BX.KLT.DINV.WD.GD.ZS,AG.YLD.CREL.KG
41,2020,3.818,0,2020,56.064731,4383953178,1.802971,7821271846,1.014463,24.222277,87.606293,1.402679,4100.768039
42,2021,4.720,1,2021,56.478131,4454512575,1.609492,7888963821,0.865486,24.477826,88.389748,2.321421,4152.232718
43,2022,4.670,0,2022,56.901955,4523589075,1.550708,7951595433,0.793914,24.748090,91.138542,1.743429,4182.382970


In [189]:
# Drop columns not needed (there is an extra "Year" column. drop glacier_extent)

df_pop_farm_parameters_2 = df_pop_farm_parameters_2.drop(["Year", "glacier_extent"], axis=1)
display(df_pop_farm_parameters_2.head(3))
df_pop_farm_parameters_2.tail(3)

,year,glacier_retreat,SP.URB.TOTL.IN.ZS,SP.URB.TOTL,SP.URB.GROW,SP.POP.TOTL,SP.POP.GROW,EN.URB.MCTY.TL.ZS,SE.PRM.CMPT.ZS,BX.KLT.DINV.WD.GD.ZS,AG.YLD.CREL.KG
0,1979,0,38.915541,1698369060,2.837103,4365802882,1.770476,16.682243,77.901047,0.428142,2331.575510
1,1980,1,39.330138,1746580426,2.838686,4442416674,1.754861,16.822785,80.020821,0.490715,2297.408450
2,1981,0,39.755159,1796677728,2.868308,4520993169,1.768778,16.929199,80.550163,0.617123,2438.721357


,year,glacier_retreat,SP.URB.TOTL.IN.ZS,SP.URB.TOTL,SP.URB.GROW,SP.POP.TOTL,SP.POP.GROW,EN.URB.MCTY.TL.ZS,SE.PRM.CMPT.ZS,BX.KLT.DINV.WD.GD.ZS,AG.YLD.CREL.KG
41,2020,0,56.064731,4383953178,1.802971,7821271846,1.014463,24.222277,87.606293,1.402679,4100.768039
42,2021,1,56.478131,4454512575,1.609492,7888963821,0.865486,24.477826,88.389748,2.321421,4152.232718
43,2022,0,56.901955,4523589075,1.550708,7951595433,0.793914,24.748090,91.138542,1.743429,4182.382970


In [190]:
# Write Parameters 2 to file
df_pop_farm_parameters_2.to_csv('resources/pop_farm_parameters_2.csv', index=False)

## Parameter Set 3:  Change in Temperature by Country (1979 to 2022)

In [191]:
# Review the dataframe
display(df_dT_country.head())
df_dT_country.tail()

,Year,AFG,ALB,DZA,ASM,AND,AGO,ATG,ARG,AUS,...,USA,VIR,URY,VUT,VEN,VNM,PSE,WLD,ZMB,ZWE
0,1972,-1.124,-0.069,-0.348,-0.067,-0.499,-0.035,-0.214,-0.008,0.091,...,-0.252,-0.130,0.250,-0.321,-0.029,-0.008,-0.505,-0.199,-0.139,-0.397
1,1973,0.232,-0.288,-0.015,0.330,0.025,0.475,0.164,-0.139,0.831,...,-0.013,0.189,-0.124,0.558,0.313,0.468,-0.479,0.269,0.383,0.601
2,1974,-0.489,-0.139,-0.503,-0.308,-0.371,-0.158,-0.377,-0.106,-0.354,...,0.081,-0.362,-0.048,0.378,-0.295,-0.387,-0.256,-0.181,-0.405,-0.485
3,1975,-0.445,-0.211,-0.539,-0.118,0.246,-0.029,-0.419,-0.021,0.048,...,-0.389,-0.240,0.031,0.239,-0.300,0.199,-0.409,0.088,-0.127,-0.262
4,1976,-0.286,-0.683,-0.782,-0.177,-0.045,-0.313,-0.467,-0.321,-0.522,...,-0.124,-0.387,-0.334,0.041,-0.409,-0.551,-0.573,-0.314,-0.314,-0.436


,Year,AFG,ALB,DZA,ASM,AND,AGO,ATG,ARG,AUS,...,USA,VIR,URY,VUT,VEN,VNM,PSE,WLD,ZMB,ZWE
46,2018,1.544,2.028,1.210,1.189,1.919,1.395,0.627,0.878,1.129,...,1.276,0.722,1.196,0.613,0.843,0.817,2.007,1.290,0.648,0.453
47,2019,0.910,1.675,1.115,1.539,1.964,1.752,0.797,0.760,1.422,...,1.034,0.898,0.766,0.897,1.235,1.811,1.204,1.444,0.855,0.925
48,2020,0.498,1.498,1.926,1.430,2.562,1.162,1.131,1.123,1.416,...,1.324,1.320,0.890,1.226,1.350,1.477,1.455,1.711,0.891,0.389
49,2021,1.327,1.536,2.330,1.268,1.533,1.553,0.862,1.031,0.629,...,1.144,0.922,0.790,1.147,0.734,1.114,1.787,1.447,0.822,-0.125
50,2022,2.012,1.518,1.688,1.256,3.243,1.212,0.770,0.643,0.754,...,1.217,0.894,0.382,1.479,0.533,1.033,1.074,1.394,0.686,-0.490


In [192]:
# Review the data types associated with the columns
df_dT_country.dtypes

Year      int64
AFG     float64
ALB     float64
DZA     float64
ASM     float64
         ...   
VNM     float64
PSE     float64
WLD     float64
ZMB     float64
ZWE     float64
Length: 171, dtype: object

In [193]:
# Check for outliers and similar anomolies
df_dT_country.describe().to_csv('resources/describe_output_dT_country.csv')
df_dT_country.describe()

,Year,AFG,ALB,DZA,ASM,AND,AGO,ATG,ARG,AUS,...,USA,VIR,URY,VUT,VEN,VNM,PSE,WLD,ZMB,ZWE
count,51.000000,51.000000,51.000000,51.000000,48.000000,51.000000,51.000000,50.00000,51.000000,51.000000,...,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000
mean,1997.000000,0.591412,0.558824,0.864216,0.611729,0.951216,0.594882,0.48210,0.385863,0.594529,...,0.618471,0.516471,0.455725,0.443882,0.489745,0.515941,0.394961,0.696412,0.501235,0.241373
std,14.866069,0.675557,0.726305,0.724532,0.497781,0.820604,0.482011,0.40125,0.366349,0.452304,...,0.564564,0.401545,0.408111,0.405994,0.431212,0.492090,0.754289,0.513082,0.435484,0.465521
min,1972.000000,-1.124000,-0.814000,-0.782000,-0.308000,-0.499000,-0.313000,-0.46700,-0.321000,-0.522000,...,-0.389000,-0.387000,-0.504000,-0.321000,-0.409000,-0.551000,-1.068000,-0.314000,-0.405000,-0.490000
25%,1984.500000,0.192000,-0.086000,0.392500,0.252500,0.238500,0.264000,0.20675,0.147500,0.294500,...,0.268000,0.195500,0.147500,0.108500,0.185000,0.123500,-0.187000,0.300000,0.152500,-0.024500
50%,1997.000000,0.498000,0.492000,0.988000,0.503000,0.946000,0.552000,0.51450,0.386000,0.604000,...,0.584000,0.541000,0.447000,0.438000,0.518000,0.416000,0.396000,0.728000,0.544000,0.170000
75%,2009.500000,1.145500,1.194500,1.285000,1.014750,1.520500,0.882500,0.76350,0.606000,0.908000,...,1.013000,0.801500,0.729500,0.697500,0.793000,0.865000,0.859000,1.037000,0.838500,0.485000
max,2022.000000,2.012000,2.028000,2.330000,1.539000,3.243000,1.752000,1.15300,1.123000,1.499000,...,2.224000,1.320000,1.546000,1.479000,1.350000,1.811000,2.326000,1.711000,1.450000,1.270000


In [194]:
# Check for non-null and null entries
display(df_dT_country.info())
display("The number of null values in the dataframe are:")
display(df_dT_country.isnull().sum().sum())
df_dT_country.isnull().sum().to_csv('resources/null_dT_country.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Columns: 171 entries, Year to ZWE
dtypes: float64(170), int64(1)
memory usage: 68.3 KB


None

'The number of null values in the dataframe are:'

81

Based upon the isnull().sum output to csv file, there are 20 columns of 170
with null values.  Delete this columns.

In [195]:
df_dT_country_r0 = df_dT_country.dropna(axis=1, how='any')
df_dT_country_r0.isnull().sum().to_csv('resources/null_dT_country_r0.csv')
df_dT_country_r0.tail(3)

,Year,AFG,ALB,DZA,AND,AGO,ARG,AUS,AUT,BHS,...,USA,VIR,URY,VUT,VEN,VNM,PSE,WLD,ZMB,ZWE
48,2020,0.498,1.498,1.926,2.562,1.162,1.123,1.416,2.315,1.611,...,1.324,1.320,0.890,1.226,1.350,1.477,1.455,1.711,0.891,0.389
49,2021,1.327,1.536,2.330,1.533,1.553,1.031,0.629,1.395,0.879,...,1.144,0.922,0.790,1.147,0.734,1.114,1.787,1.447,0.822,-0.125
50,2022,2.012,1.518,1.688,3.243,1.212,0.643,0.754,2.498,1.480,...,1.217,0.894,0.382,1.479,0.533,1.033,1.074,1.394,0.686,-0.490


In [196]:
# Filter for all rows 1979 and up
df_dT_country_r1 = df_dT_country_r0[df_dT_country_r0['Year'] >= 1979]
df_dT_country_r1 = df_dT_country_r1.reset_index(drop=True)
df_dT_country_r1.tail(3)

,Year,AFG,ALB,DZA,AND,AGO,ARG,AUS,AUT,BHS,...,USA,VIR,URY,VUT,VEN,VNM,PSE,WLD,ZMB,ZWE
41,2020,0.498,1.498,1.926,2.562,1.162,1.123,1.416,2.315,1.611,...,1.324,1.320,0.890,1.226,1.350,1.477,1.455,1.711,0.891,0.389
42,2021,1.327,1.536,2.330,1.533,1.553,1.031,0.629,1.395,0.879,...,1.144,0.922,0.790,1.147,0.734,1.114,1.787,1.447,0.822,-0.125
43,2022,2.012,1.518,1.688,3.243,1.212,0.643,0.754,2.498,1.480,...,1.217,0.894,0.382,1.479,0.533,1.033,1.074,1.394,0.686,-0.490


In [197]:
# Combine the dataframes into one dataframe
df_dT_country_parameters_3 = pd.concat([df_glacier_retreat, df_dT_country_r1], axis="columns",join="inner")
display(df_dT_country_parameters_3.head(3))
df_dT_country_parameters_3.tail(3)

,year,glacier_extent,glacier_retreat,Year,AFG,ALB,DZA,AND,AGO,ARG,...,USA,VIR,URY,VUT,VEN,VNM,PSE,WLD,ZMB,ZWE
0,1979,6.903,0,1979,0.361,0.203,0.654,0.058,0.291,0.266,...,-0.306,-0.062,0.128,0.103,0.202,0.416,0.606,0.226,0.249,0.189
1,1980,7.544,1,1980,0.600,-0.414,0.232,-0.188,0.279,0.373,...,0.412,0.718,0.571,0.147,0.346,0.368,-0.204,0.332,0.138,-0.024
2,1981,6.903,0,1981,0.483,-0.351,0.215,0.178,-0.071,0.378,...,0.871,0.514,0.500,0.099,0.134,0.490,-0.215,0.443,-0.158,-0.360


,year,glacier_extent,glacier_retreat,Year,AFG,ALB,DZA,AND,AGO,ARG,...,USA,VIR,URY,VUT,VEN,VNM,PSE,WLD,ZMB,ZWE
41,2020,3.818,0,2020,0.498,1.498,1.926,2.562,1.162,1.123,...,1.324,1.320,0.890,1.226,1.350,1.477,1.455,1.711,0.891,0.389
42,2021,4.720,1,2021,1.327,1.536,2.330,1.533,1.553,1.031,...,1.144,0.922,0.790,1.147,0.734,1.114,1.787,1.447,0.822,-0.125
43,2022,4.670,0,2022,2.012,1.518,1.688,3.243,1.212,0.643,...,1.217,0.894,0.382,1.479,0.533,1.033,1.074,1.394,0.686,-0.490


In [198]:
# Drop columns not needed (there is an extra "Year" column. drop glacier_extent)

df_dT_country_parameters_3 = df_dT_country_parameters_3.drop(["Year", "glacier_extent"], axis=1)
display(df_dT_country_parameters_3.head(3))
df_dT_country_parameters_3.tail(3)

,year,glacier_retreat,AFG,ALB,DZA,AND,AGO,ARG,AUS,AUT,...,USA,VIR,URY,VUT,VEN,VNM,PSE,WLD,ZMB,ZWE
0,1979,0,0.361,0.203,0.654,0.058,0.291,0.266,0.375,-0.112,...,-0.306,-0.062,0.128,0.103,0.202,0.416,0.606,0.226,0.249,0.189
1,1980,1,0.600,-0.414,0.232,-0.188,0.279,0.373,0.887,-0.274,...,0.412,0.718,0.571,0.147,0.346,0.368,-0.204,0.332,0.138,-0.024
2,1981,0,0.483,-0.351,0.215,0.178,-0.071,0.378,0.495,0.277,...,0.871,0.514,0.500,0.099,0.134,0.490,-0.215,0.443,-0.158,-0.360


,year,glacier_retreat,AFG,ALB,DZA,AND,AGO,ARG,AUS,AUT,...,USA,VIR,URY,VUT,VEN,VNM,PSE,WLD,ZMB,ZWE
41,2020,0,0.498,1.498,1.926,2.562,1.162,1.123,1.416,2.315,...,1.324,1.320,0.890,1.226,1.350,1.477,1.455,1.711,0.891,0.389
42,2021,1,1.327,1.536,2.330,1.533,1.553,1.031,0.629,1.395,...,1.144,0.922,0.790,1.147,0.734,1.114,1.787,1.447,0.822,-0.125
43,2022,0,2.012,1.518,1.688,3.243,1.212,0.643,0.754,2.498,...,1.217,0.894,0.382,1.479,0.533,1.033,1.074,1.394,0.686,-0.490


In [199]:
# Write Parameters 3 to file
df_dT_country_parameters_3.to_csv('resources/dT_Country_parameters_3.csv', index=False)

## Parameter Set 4:  Change in Forestation by Country (1991 to 2021)

In [200]:
# Review the dataframe
display(df_forestation.head())
df_forestation.tail()

,Year,ABW,AFE,AFG,AFW,AGO,ALB,AND,ARB,ARE,...,VIR,VNM,VUT,WLD,WSM,XKX,YEM,ZAF,ZMB,ZWE
0,1991,0,-0.151253,0,-0.118162,-0.124663,-0.071168,0,-0.018069,0.090721,...,-1.162857,0.739851,0,0.000000,-0.170318,0,0,-0.030006,-0.048158,-0.11909
1,1992,0,-0.151253,0,-0.118162,-0.124663,-0.071168,0,-0.018074,0.090721,...,-1.162857,0.739851,0,32.511019,-0.170318,0,0,-0.030006,-0.048158,-0.11909
2,1993,0,-0.151253,0,-0.118162,-0.124663,-0.071168,0,-0.018069,0.090721,...,-1.162857,0.739851,0,-0.056167,-0.170318,0,0,-0.030006,-0.048158,-0.11909
3,1994,0,-0.151253,0,-0.118162,-0.124663,-0.071168,0,-0.018069,0.090721,...,-1.162857,0.739851,0,-0.057699,-0.170318,0,0,-0.030006,-0.048158,-0.11909
4,1995,0,-0.151253,0,-0.118162,-0.124663,-0.071168,0,-0.018069,0.090721,...,-1.162857,0.739851,0,-0.059895,-0.170318,0,0,-0.030006,-0.048158,-0.11909


,Year,ABW,AFE,AFG,AFW,AGO,ALB,AND,ARB,ARE,...,VIR,VNM,VUT,WLD,WSM,XKX,YEM,ZAF,ZMB,ZWE
26,2017,0,-0.215365,0,-0.081283,-0.445215,-0.010128,0,-0.019380,0.0,...,0.428571,0.117982,0,-0.050919,0.867008,0,0,-0.030006,-0.253070,-0.11909
27,2018,0,-0.212804,0,-0.085228,-0.445223,-0.000091,0,-0.019522,0.0,...,0.428571,0.238345,0,-0.037361,-0.172662,0,0,-0.030006,-0.253178,-0.11909
28,2019,0,-0.219886,0,-0.084885,-0.445231,0.000000,0,-0.017800,0.0,...,0.428571,0.242160,0,-0.038881,-0.172662,0,0,-0.030006,-0.253191,-0.11909
29,2020,0,-0.217373,0,-0.084437,-0.445223,0.000000,0,-0.017726,0.0,...,0.428571,0.242160,0,-0.036850,-0.172662,0,0,-0.030006,-0.253178,-0.11909
30,2021,0,-0.219059,0,-0.084741,-0.445229,0.000000,0,-0.017774,0.0,...,0.428571,0.242160,0,0.000144,-0.172662,0,0,-0.030006,-0.253187,-0.11909


In [201]:
# Review the data types associated with the columns
df_forestation.dtypes

Year      int64
ABW       int64
AFE     float64
AFG       int64
AFW     float64
         ...   
XKX       int64
YEM       int64
ZAF     float64
ZMB     float64
ZWE     float64
Length: 267, dtype: object

In [202]:
# Check for outliers and similar anomolies
df_forestation.describe().to_csv('resources/describe_output_df_forestation.csv')
df_forestation.describe()

,Year,ABW,AFE,AFG,AFW,AGO,ALB,AND,ARB,ARE,...,VIR,VNM,VUT,WLD,WSM,XKX,YEM,ZAF,ZMB,ZWE
count,31.000000,31.0,31.000000,31.0,31.000000,31.000000,31.000000,31.0,31.000000,31.000000,...,31.000000,31.000000,31.0,31.000000,31.000000,31.0,31.0,3.100000e+01,31.000000,3.100000e+01
mean,2006.000000,0.0,-0.202413,0.0,-0.098221,-0.341818,0.000118,0.0,-0.030343,0.032839,...,-0.412903,0.585661,0.0,1.005711,-0.137250,0.0,0.0,-3.000600e-02,-0.120902,-1.190900e-01
std,9.092121,0.0,0.084250,0.0,0.014324,0.152328,0.053531,0.0,0.074822,0.040866,...,0.673461,0.400637,0.0,5.847166,0.186384,0.0,0.0,1.763399e-17,0.099709,5.642875e-17
min,1991.000000,0.0,-0.523168,0.0,-0.118162,-0.445231,-0.071168,0.0,-0.433371,0.000000,...,-1.162857,0.117982,0.0,-0.077903,-0.172662,0.0,0.0,-3.000600e-02,-0.253299,-1.190900e-01
25%,1998.500000,0.0,-0.212613,0.0,-0.118162,-0.445225,-0.071168,0.0,-0.018069,0.000000,...,-1.162857,0.417204,0.0,-0.056933,-0.171025,0.0,0.0,-3.000600e-02,-0.253138,-1.190900e-01
50%,2006.000000,0.0,-0.179715,0.0,-0.092127,-0.445224,0.000000,0.0,-0.018069,0.011081,...,-0.582857,0.517290,0.0,-0.039767,-0.170318,0.0,0.0,-3.000600e-02,-0.048158,-1.190900e-01
75%,2013.500000,0.0,-0.151253,0.0,-0.087956,-0.124663,0.046606,0.0,-0.014417,0.090721,...,0.417143,0.739851,0.0,-0.035685,-0.170318,0.0,0.0,-3.000600e-02,-0.048158,-1.190900e-01
max,2021.000000,0.0,-0.151253,0.0,-0.081283,-0.124663,0.051953,0.0,-0.013782,0.090721,...,0.428571,2.419354,0.0,32.511019,0.867008,0.0,0.0,-3.000600e-02,-0.048158,-1.190900e-01


In [203]:
# Check for non-null and null entries
display(df_forestation.info())
display("The number of null values in the dataframe are:")
display(df_forestation.isnull().sum().sum())
df_forestation.isnull().sum().to_csv('resources/null_df_forestation.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Columns: 267 entries, Year to ZWE
dtypes: float64(236), int64(31)
memory usage: 64.8 KB


None

'The number of null values in the dataframe are:'

0

In [204]:
# There are no null values but there are columns with all zero values.
# Delete these columns using loc function
df_forestation_r0 = df_forestation.loc[:, (df_forestation.sum() != 0)]
df_forestation_r0.tail(3)

,Year,AFE,AFW,AGO,ALB,ARB,ARE,ARG,ARM,ASM,...,VCT,VEN,VGB,VIR,VNM,WLD,WSM,ZAF,ZMB,ZWE
28,2019,-0.219886,-0.084885,-0.445231,0.0,-0.017800,0.0,-0.040195,-0.007376,-0.15,...,0.0,-0.074554,0.0,0.428571,0.24216,-0.038881,-0.172662,-0.030006,-0.253191,-0.11909
29,2020,-0.217373,-0.084437,-0.445223,0.0,-0.017726,0.0,-0.039464,-0.007376,-0.15,...,0.0,-0.046596,0.0,0.428571,0.24216,-0.036850,-0.172662,-0.030006,-0.253178,-0.11909
30,2021,-0.219059,-0.084741,-0.445229,0.0,-0.017774,0.0,-0.039951,-0.007376,-0.15,...,0.0,-0.065234,0.0,0.428571,0.24216,0.000144,-0.172662,-0.030006,-0.253187,-0.11909


In [205]:
# Some columns appear to not change, although this could be correct. 
# However, columns that don't change also don't impact change in glacier retreat.
# Reference:  The following code provided by ChatGPT
# Prompt:  "what are examples of python code that identify a column of dataframe 
#   has values that are not changing and should be removed from the dataframe"
constant_columns = [col for col in df_forestation_r0.columns if df_forestation_r0[col].nunique() == 1]
print(constant_columns)

['BWA', 'CAF', 'LBR', 'MWI', 'SLV', 'SOM', 'ZAF', 'ZWE']


In [206]:
# Drop columns with unchanging data

df_forestation_r1 = df_forestation_r0.drop(['BWA', 'CAF', 'LBR', 'MWI', 'SLV', 'SOM', 'ZAF', 'ZWE'], axis=1)
display(df_forestation_r1.head(3))
df_forestation_r1.tail(3)

,Year,AFE,AFW,AGO,ALB,ARB,ARE,ARG,ARM,ASM,...,USA,UZB,VCT,VEN,VGB,VIR,VNM,WLD,WSM,ZMB
0,1991,-0.151253,-0.118162,-0.124663,-0.071168,-0.018069,0.090721,-0.066723,0.000000,-0.17,...,0.011857,0.000000,0.25641,-0.325945,-0.026667,-1.162857,0.739851,0.000000,-0.170318,-0.048158
1,1992,-0.151253,-0.118162,-0.124663,-0.071168,-0.018074,0.090721,-0.066723,11.742606,-0.17,...,0.011857,6.186621,0.25641,-0.325945,-0.026667,-1.162857,0.739851,32.511019,-0.170318,-0.048158
2,1993,-0.151253,-0.118162,-0.124663,-0.071168,-0.018069,0.090721,-0.066723,-0.007341,-0.17,...,0.011857,0.096871,0.25641,-0.325945,-0.026667,-1.162857,0.739851,-0.056167,-0.170318,-0.048158


,Year,AFE,AFW,AGO,ALB,ARB,ARE,ARG,ARM,ASM,...,USA,UZB,VCT,VEN,VGB,VIR,VNM,WLD,WSM,ZMB
28,2019,-0.219886,-0.084885,-0.445231,0.0,-0.017800,0.0,-0.040195,-0.007376,-0.15,...,0.0,0.059253,0.0,-0.074554,0.0,0.428571,0.24216,-0.038881,-0.172662,-0.253191
29,2020,-0.217373,-0.084437,-0.445223,0.0,-0.017726,0.0,-0.039464,-0.007376,-0.15,...,0.0,0.056848,0.0,-0.046596,0.0,0.428571,0.24216,-0.036850,-0.172662,-0.253178
30,2021,-0.219059,-0.084741,-0.445229,0.0,-0.017774,0.0,-0.039951,-0.007376,-0.15,...,0.0,0.058583,0.0,-0.065234,0.0,0.428571,0.24216,0.000144,-0.172662,-0.253187


In [207]:
# The y output glacier retreat data starts in 1979.  Remove rows greater than
# 1991 prior to concat
# Filter for all rows 1991 and up
df_glacier_retreat_r0 = df_glacier_retreat[df_glacier_retreat['year'] >= 1991]
# Also drop 2022 and 2023
df_glacier_retreat_r1 = df_glacier_retreat_r0[df_glacier_retreat_r0['year'] <= 2021]
df_glacier_retreat_r1 = df_glacier_retreat_r1.reset_index(drop=True)

display(df_glacier_retreat_r1.head(3))
df_glacier_retreat_r1.tail(3)

,year,glacier_extent,glacier_retreat
0,1991,6.302,1
1,1992,7.209,1
2,1993,6.184,0


,year,glacier_extent,glacier_retreat
28,2019,4.192,0
29,2020,3.818,0
30,2021,4.720,1


In [208]:
# Combine the dataframes into one dataframe
df_dForest_country_parameters_4 = pd.concat([df_glacier_retreat_r1, df_forestation_r1], axis="columns",join="inner")
display(df_dForest_country_parameters_4.head(3))
df_dForest_country_parameters_4.tail(3)

,year,glacier_extent,glacier_retreat,Year,AFE,AFW,AGO,ALB,ARB,ARE,...,USA,UZB,VCT,VEN,VGB,VIR,VNM,WLD,WSM,ZMB
0,1991,6.302,1,1991,-0.151253,-0.118162,-0.124663,-0.071168,-0.018069,0.090721,...,0.011857,0.000000,0.25641,-0.325945,-0.026667,-1.162857,0.739851,0.000000,-0.170318,-0.048158
1,1992,7.209,1,1992,-0.151253,-0.118162,-0.124663,-0.071168,-0.018074,0.090721,...,0.011857,6.186621,0.25641,-0.325945,-0.026667,-1.162857,0.739851,32.511019,-0.170318,-0.048158
2,1993,6.184,0,1993,-0.151253,-0.118162,-0.124663,-0.071168,-0.018069,0.090721,...,0.011857,0.096871,0.25641,-0.325945,-0.026667,-1.162857,0.739851,-0.056167,-0.170318,-0.048158


,year,glacier_extent,glacier_retreat,Year,AFE,AFW,AGO,ALB,ARB,ARE,...,USA,UZB,VCT,VEN,VGB,VIR,VNM,WLD,WSM,ZMB
28,2019,4.192,0,2019,-0.219886,-0.084885,-0.445231,0.0,-0.017800,0.0,...,0.0,0.059253,0.0,-0.074554,0.0,0.428571,0.24216,-0.038881,-0.172662,-0.253191
29,2020,3.818,0,2020,-0.217373,-0.084437,-0.445223,0.0,-0.017726,0.0,...,0.0,0.056848,0.0,-0.046596,0.0,0.428571,0.24216,-0.036850,-0.172662,-0.253178
30,2021,4.720,1,2021,-0.219059,-0.084741,-0.445229,0.0,-0.017774,0.0,...,0.0,0.058583,0.0,-0.065234,0.0,0.428571,0.24216,0.000144,-0.172662,-0.253187


In [209]:
# Drop columns not needed (there is an extra "Year" column. drop glacier_extent)

df_dForest_country_parameters_4 = df_dForest_country_parameters_4.drop(["Year", "glacier_extent"], axis=1)
display(df_dForest_country_parameters_4.head(3))
df_dForest_country_parameters_4.tail(3)

,year,glacier_retreat,AFE,AFW,AGO,ALB,ARB,ARE,ARG,ARM,...,USA,UZB,VCT,VEN,VGB,VIR,VNM,WLD,WSM,ZMB
0,1991,1,-0.151253,-0.118162,-0.124663,-0.071168,-0.018069,0.090721,-0.066723,0.000000,...,0.011857,0.000000,0.25641,-0.325945,-0.026667,-1.162857,0.739851,0.000000,-0.170318,-0.048158
1,1992,1,-0.151253,-0.118162,-0.124663,-0.071168,-0.018074,0.090721,-0.066723,11.742606,...,0.011857,6.186621,0.25641,-0.325945,-0.026667,-1.162857,0.739851,32.511019,-0.170318,-0.048158
2,1993,0,-0.151253,-0.118162,-0.124663,-0.071168,-0.018069,0.090721,-0.066723,-0.007341,...,0.011857,0.096871,0.25641,-0.325945,-0.026667,-1.162857,0.739851,-0.056167,-0.170318,-0.048158


,year,glacier_retreat,AFE,AFW,AGO,ALB,ARB,ARE,ARG,ARM,...,USA,UZB,VCT,VEN,VGB,VIR,VNM,WLD,WSM,ZMB
28,2019,0,-0.219886,-0.084885,-0.445231,0.0,-0.017800,0.0,-0.040195,-0.007376,...,0.0,0.059253,0.0,-0.074554,0.0,0.428571,0.24216,-0.038881,-0.172662,-0.253191
29,2020,0,-0.217373,-0.084437,-0.445223,0.0,-0.017726,0.0,-0.039464,-0.007376,...,0.0,0.056848,0.0,-0.046596,0.0,0.428571,0.24216,-0.036850,-0.172662,-0.253178
30,2021,1,-0.219059,-0.084741,-0.445229,0.0,-0.017774,0.0,-0.039951,-0.007376,...,0.0,0.058583,0.0,-0.065234,0.0,0.428571,0.24216,0.000144,-0.172662,-0.253187


In [210]:
# Write Parameters 4 to file
df_dForest_country_parameters_4.to_csv('resources/deforest_parameters_4.csv', index=False)

### End data cleaning